# 🛒 Superstore Sales: Merging, Concatenation, Pivoting & Melting

This notebook demonstrates:
- `pd.merge()` for joining tables
- `pd.concat()` for stacking
- `pivot_table()` and `melt()` for reshaping


## 📥 Step 1: Load Superstore Dataset

In [1]:
import pandas as pd

# url = "https://raw.githubusercontent.com/nileshely/SuperStore-Dataset-2019-2022/main/superstore_dataset.csv"
#df = pd.read_csv(url)
#df.to_csv("superstore.csv") #save dataset from url to current working directory
df = pd.read_csv('superstore.csv')

print(df.columns)
df.head()

Index(['Unnamed: 0', 'order_id', 'order_date', 'ship_date', 'customer',
       'manufactory', 'product_name', 'segment', 'category', 'subcategory',
       'region', 'zip', 'city', 'state', 'country', 'discount', 'profit',
       'quantity', 'sales', 'profit_margin'],
      dtype='object')


,Unnamed: 0,order_id,order_date,ship_date,customer,manufactory,product_name,segment,category,subcategory,region,zip,city,state,country,discount,profit,quantity,sales,profit_margin
0,0,US-2020-103800,1/3/2019,1/7/2019,Darren Powers,Message Book,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",Consumer,Office Supplies,Paper,Central,77095,Houston,Texas,United States,0.2,5.5512,2,16.448,0.3375
1,1,US-2020-112326,1/4/2019,1/8/2019,Phillina Ober,GBC,GBC Standard Plastic Binding Systems Combs,Home Office,Office Supplies,Binders,Central,60540,Naperville,Illinois,United States,0.8,-5.4870,2,3.540,-1.5500
2,2,US-2020-112326,1/4/2019,1/8/2019,Phillina Ober,Avery,Avery 508,Home Office,Office Supplies,Labels,Central,60540,Naperville,Illinois,United States,0.2,4.2717,3,11.784,0.3625
3,3,US-2020-112326,1/4/2019,1/8/2019,Phillina Ober,SAFCO,SAFCO Boltless Steel Shelving,Home Office,Office Supplies,Storage,Central,60540,Naperville,Illinois,United States,0.2,-64.7748,3,272.736,-0.2375
4,4,US-2020-141817,1/5/2019,1/12/2019,Mick Brown,Avery,Avery Hi-Liter EverBold Pen Style Fluorescent ...,Consumer,Office Supplies,Art,East,19143,Philadelphia,Pennsylvania,United States,0.2,4.8840,3,19.536,0.2500


## 🧱 Step 2: Create Tables for Merging

To prepare for merge operations, we extract a few DataFrames:

- orders_df: A deduplicated view of relevant order data, containing key   attributes like order ID, customer, sales, and region.

- returns_df: A simulated dataset indicating returned orders. About 30% of unique order IDs are sampled and marked with a Returned = True column.

- people_df: A fictional assignment of managers to each unique region. This prepares us for region-based enrichment of the order data.

In [3]:
orders_df = df[['order_id', 'order_date', 'customer', 'sales', 'region', 'segment']].drop_duplicates()

returns_df = pd.DataFrame({
    'order_id': df['order_id'].sample(frac=0.3, random_state=42).unique(),
    'Returned': True
})

print("Shape of the Returns_df")
print(returns_df.shape)

people_df = df[['region']].drop_duplicates().assign(Manager=['Alice', 'Bob', 'Clara', 'David'])
orders_df.head(), returns_df.head(), people_df.head()

Shape of the Returns_df
(2266, 2)


(         order_id order_date       customer    sales   region      segment
 0  US-2020-103800   1/3/2019  Darren Powers   16.448  Central     Consumer
 1  US-2020-112326   1/4/2019  Phillina Ober    3.540  Central  Home Office
 2  US-2020-112326   1/4/2019  Phillina Ober   11.784  Central  Home Office
 3  US-2020-112326   1/4/2019  Phillina Ober  272.736  Central  Home Office
 4  US-2020-141817   1/5/2019     Mick Brown   19.536     East     Consumer,
          order_id  Returned
 0  US-2021-145849      True
 1  US-2020-165764      True
 2  US-2022-112669      True
 3  US-2020-146591      True
 4  US-2022-152800      True,
     region Manager
 0  Central   Alice
 4     East     Bob
 5    South   Clara
 6     West   David)

## 🔁 Step 3: Merge (Inner Join)

### Inner Merge
An inner merge between orders_df and returns_df returns only the records with matching order_ids in both DataFrames. This helps us identify which specific orders were returned, excluding any unmatched entries. It’s commonly used when we want to work only with overlapping data.

In [5]:
merged_inner = pd.merge(orders_df, returns_df, on='order_id', how='inner')
merged_inner.head()


print(orders_df.shape, returns_df.shape, merged_inner.shape)

(9992, 6) (2266, 2) (5717, 7)


## 🔄 Step 4: Merge (Left Join)

### Left Merge with Missing Value Handling
The left merge includes all records from the orders(ie. the left table) and only the matching rows from returns. Orders without a return get a NaN in the Returned column. To make this binary and clean, we replace these missing values with False. This technique is useful when we want to preserve the full order list and flag returns where applicable.

In [6]:
merged_left = pd.merge(orders_df, returns_df, on='order_id', how='left')
merged_left['Returned'] = merged_left['Returned'].fillna(False)
merged_left.head()

print("shape of the merged left and the ordered df")
print(merged_left.shape, orders_df.shape)


shape of the merged left and the ordered df
(9992, 7) (9992, 6)


C:\Users\JOHN PAUL\AppData\Local\Temp\ipykernel_6784\848815336.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_left['Returned'] = merged_left['Returned'].fillna(False)


## 🌐 Step 5: Merge (Outer Join)

### Outer Merge with Indicator
An outer merge combines all rows from both DataFrames, whether they match or not. The special indicator=True argument adds a _merge column, showing whether each row came from the left (orders), right (returns), or both. This is ideal for comparing datasets or understanding overlap.

In [7]:
merged_outer = pd.merge(orders_df, returns_df, on='order_id', how='outer', indicator=True)
merged_outer.head(50)

,order_id,order_date,customer,sales,region,segment,Returned,_merge
0,US-2020-100006,9/7/2019,Dennis Kane,377.970,East,Consumer,NaN,left_only
1,US-2020-100090,7/8/2019,Ed Braxton,502.488,West,Corporate,NaN,left_only
2,US-2020-100090,7/8/2019,Ed Braxton,196.704,West,Corporate,NaN,left_only
3,US-2020-100279,3/10/2019,Scott Williamson,22.380,Central,Consumer,NaN,left_only
4,US-2020-100293,3/14/2019,Neil Französisch,91.056,South,Home Office,NaN,left_only
5,US-2020-100328,1/28/2019,Jasper Cacioppo,3.928,East,Consumer,NaN,left_only
6,US-2020-100363,4/8/2019,Jim Mitchum,2.368,West,Corporate,True,both
7,US-2020-100363,4/8/2019,Jim Mitchum,19.008,West,Corporate,True,both
8,US-2020-100391,5/25/2019,Barry Weirich,14.620,East,Consumer,True,both
9,US-2020-100678,4/18/2019,Kunst Miller,317.058,Central,Consumer,True,both


## 📊 Step 6: Concatenation (Vertical Stack)

### Concatenation (Vertical Stacking)
We split the orders_df into two halves using sample() and drop(), and then stack them back together using pd.concat(). This is a simple demonstration of combining datasets vertically—useful when appending rows from different periods, sources, or files with the same schema.

In [9]:
q1 = orders_df.sample(frac=0.5, random_state=1)
q2 = orders_df.drop(q1.index)
vertical_stack = pd.concat([q1, q2], axis=0)
vertical_stack.head(50)

# print(q1.shape, q2.shape)

,order_id,order_date,customer,sales,region,segment
1816,US-2020-116568,12/14/2019,Bryan Mills,186.304,South,Consumer
883,US-2020-109897,8/12/2019,Ben Wallace,806.336,West,Consumer
1124,US-2020-106992,9/19/2019,Sean Braxton,3059.982,Central,Corporate
6397,US-2022-109722,12/5/2021,Theone Pippenger,699.980,South,Consumer
2913,US-2021-128090,8/16/2020,John Murray,2.304,West,Consumer
6607,US-2022-157868,12/23/2021,Matt Collister,24.850,Central,Corporate
3061,US-2021-117772,9/10/2020,Matt Collins,353.880,West,Consumer
3835,US-2021-114741,12/6/2020,Ivan Liston,1325.850,West,Consumer
278,US-2020-128055,3/31/2019,Alex Avila,673.568,West,Consumer
915,US-2020-114321,8/20/2019,Nick Crebassa,896.990,South,Corporate


## ➕ Step 7: Concatenation (Horizontal Stack)

### Merging with People (Managers)
We enrich orders_df by merging it with people_df, using the common region column. This brings in manager information per region, useful for reporting, accountability, or filtering. It illustrates how merging can add descriptive attributes to transactional data.

In [12]:
orders_plus_manager = pd.merge(orders_df, people_df, on='region', how='left')
orders_plus_manager.head(40)

,order_id,order_date,customer,sales,region,segment,Manager
0,US-2020-103800,1/3/2019,Darren Powers,16.448,Central,Consumer,Alice
1,US-2020-112326,1/4/2019,Phillina Ober,3.540,Central,Home Office,Alice
2,US-2020-112326,1/4/2019,Phillina Ober,11.784,Central,Home Office,Alice
3,US-2020-112326,1/4/2019,Phillina Ober,272.736,Central,Home Office,Alice
4,US-2020-141817,1/5/2019,Mick Brown,19.536,East,Consumer,Bob
5,US-2020-106054,1/6/2019,Jack O'Briant,12.780,South,Corporate,Clara
6,US-2020-130813,1/6/2019,Lycoris Saunders,19.440,West,Consumer,David
7,US-2020-167199,1/6/2019,Maria Etezadi,2573.820,South,Home Office,Clara
8,US-2020-167199,1/6/2019,Maria Etezadi,5.480,South,Home Office,Clara
9,US-2020-167199,1/6/2019,Maria Etezadi,609.980,South,Home Office,Clara


## 🔄 Step 8: Pivoting and Melting DataFrames

### Pivot Table: Sales by Region and Segment
Pivot tables allow you to summarize your data across dimensions. Here, we use region as the row index and segment as columns to aggregate sales via sum. This is useful for comparing performance across different segments and regions in a clean table format.

In [18]:
pivot = df.pivot_table(index='region', columns='segment', values='sales', aggfunc='sum')
pivot

segment,Consumer,Corporate,Home Office
region,,,
Central,252031.434,157995.8128,91212.6440
East,350908.167,200409.3470,127463.7260
South,195580.971,121885.9325,74255.0015
West,362880.773,225855.2745,136721.7770


### 🧭 Navigating MultiIndex

### Multi-Index Pivot Table
This pivot table uses two index levels—region and category—and summarizes sales. It’s ideal for hierarchical reporting, where you want to break down data into sub-groups. This structure helps in identifying trends and performance within both broad and specific categories.

In [20]:
multi = df.pivot_table(index=['region', 'category'], values='sales', aggfunc='sum')
multi.head(50)

sales
region  category                    
Central Furniture        163797.1638
        Office Supplies  167026.4150
        Technology       170416.3120
East    Furniture        208291.2040
        Office Supplies  205516.0550
        Technology       264973.9810
South   Furniture        117298.6840
        Office Supplies  125651.3130
        Technology       148771.9080
West    Furniture        252612.7435
        Office Supplies  220853.2490
        Technology       251991.8320

## 🧯 Step 9: Melting DataFrames

### Melting DataFrames (Unpivoting)
The melt() function is used to transform a wide DataFrame into a long one, often called “unpivoting.” In this case, the pivot table showing sales by region and segment is first reset using .reset_index() to turn the index (region) back into a column. Then, melt() is applied to convert the wide-format columns (Consumer, Corporate, etc.) into two columns: one for the variable name (segment) and another for the values (sales). This format is more suitable for plotting or further analysis, as it aligns with tidy data principles—where each row is a single observation.

In [21]:
melted = pivot.reset_index().melt(id_vars='region', var_name='segment', value_name='sales')
melted.head()

,region,segment,sales
0,Central,Consumer,252031.4340
1,East,Consumer,350908.1670
2,South,Consumer,195580.9710
3,West,Consumer,362880.7730
4,Central,Corporate,157995.8128


## ✅ Summary of All Concepts


We practiced:
- `pd.merge()` for relational joins
- `pd.concat()` for stacking DataFrames
- `pivot_table()` for summarized, multi-index views
- `melt()` for reversing wide-to-long format

These techniques reshape and organize your data for insights and presentation.
